In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.5/112.6 GB disk)


In [ ]:
!pip install optuna
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="bSbuPJyAkojNGnXNQwV6")
project = rf.workspace("detectionclassificationgarbage").project("detection-garbage-jkww2")
version = project.version(5)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Detection-Garbage-5 in yolov11:: 100%|██████████| 45348/45348 [00:07<00:00, 6237.81it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
db_path = "sqlite:////content/drive/MyDrive/SIC/capstone/optuna.db"

## YOLO

In [ ]:
import optuna
from ultralytics import YOLO
import os

def objective_yolo(trial):
    lr = trial.suggest_float("lr0", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)

    model = YOLO("yolo11l.pt")
    model.train(
        data="/content/Detection-Garbage-5/data.yaml",
        epochs=2,
        imgsz=640,
        lr0=lr,
        weight_decay=weight_decay,
        optimizer="AdamW",
        project="optuna_yolo",
        name=f"trial_{trial.number}",
        exist_ok=True,
        verbose=False
    )

    metrics = model.val()
    map50 = metrics.box.map50

    return map50


In [ ]:
study = optuna.create_study(
    study_name="yolo_tuning",
    storage=db_path,
    direction="maximize",
    load_if_exists=True
)
study.optimize(objective_yolo, n_trials=1)

print("Best trial:")
print(study.best_trial.params)


[I 2025-06-25 03:48:33,105] Using an existing study with name 'yolo_tuning' instead of creating a new one.


100%|██████████| 49.0M/49.0M [00:00<00:00, 314MB/s]


Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Detection-Garbage-5/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0008380253100305082, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=trial_5, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, plots

100%|██████████| 755k/755k [00:00<00:00, 20.5MB/s]

Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytics

...

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 16, len(boxes) = 6235. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 121/121 [01:14<00:00,  1.63it/s]


                   all       1935       6235      0.401      0.386      0.345       0.23
Speed: 0.3ms preprocess, 33.4ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to optuna_yolo/trial_5


[I 2025-06-25 04:24:40,618] Trial 5 finished with value: 0.34460117703535775 and parameters: {'lr0': 0.0008380253100305082, 'weight_decay': 0.000198009586978084}. Best is trial 3 with value: 0.5704894636282879.


Best trial:
{'lr0': 1.1102663489094835e-05, 'weight_decay': 2.3239449479655278e-06}


In [ ]:
study = optuna.load_study(
    study_name="yolo_tuning",
    storage=db_path
)

best_trial = study.best_trial

print("✅ Giá trị mục tiêu tốt nhất:", best_trial.value)
print("✅ Bộ tham số tối ưu:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

✅ Giá trị mục tiêu tốt nhất: 0.5704894636282879
✅ Bộ tham số tối ưu:
  lr0: 1.1102663489094835e-05
  weight_decay: 2.3239449479655278e-06


In [ ]:
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_param_importances(study).show()
